# Overview of Missions 

In [2]:
import cftime
import glob
import matplotlib.pyplot as plt
import mpl_scatter_density
import netCDF4 as nc
import numpy as np
import pandas as pd
from scipy import stats
from scipy import io

import mission_tools

In [3]:
mission_list = glob.glob('../data/microswift_missions/mission_*.nc')
num_waves_in_mission = []
mission_start_time = []
mission_end_time = []
mission_median_time = []
offshore_wave_height = []
offshore_wave_period = []
offshore_wave_direction = []
mission_num = []

for mission_nc in mission_list:
    mission_dataset = nc.Dataset(mission_nc, mode='r')
    x_locations = mission_dataset['xFRF'][:]
    y_locations = mission_dataset['yFRF'][:]
    eta = mission_dataset['sea_surface_elevation'][:]
    time = mission_dataset['time'][:]
    time_units = mission_dataset['time'].units
    time_calendar = mission_dataset['time'].calendar
    median_mission_time = np.median(time)

    wave_heights, \
    _, \
    _, \
    wave_times = mission_tools.compute_individual_waves(x_locations, y_locations,
                                                        eta, time, time_units,
                                                        time_calendar,
                                                        single_trajectory=False )
    
    num_waves_in_mission.append(len(wave_heights))
    mission_num.append(int(mission_nc[36:-3]))
    mission_start_time.append(time[0])
    mission_end_time.append(time[-1])
    mission_median_time.append(np.median(time))
    
    # Find Offshore Wave Height and Period During the Mission
    offshore_wave_height.append(mission_tools.closest_awac_sig_wave_height(np.median(time), 
                                '../data/FRFdata/FRF-ocean_waves_8m-array_202110.nc'))
    offshore_wave_period.append(mission_tools.closest_awac_mean_wave_period(np.median(time), 
                                '../data/FRFdata/FRF-ocean_waves_8m-array_202110.nc'))
    offshore_wave_direction.append(mission_tools.closest_awac_mean_wave_direction(np.median(time), 
                                '../data/FRFdata/FRF-ocean_waves_8m-array_202110.nc'))

/Users/edwinrainville/opt/miniconda3/envs/dissip_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:784: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/edwinrainville/opt/miniconda3/envs/dissip_env/lib/python3.9/site-packages/numpy/ma/core.py:5305: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
/Users/edwinrainville/opt/miniconda3/envs/dissip_env/lib/python3.9/site-packages/numpy/core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [4]:
# Create Dataframe of mission Metadata
mission_df = pd.DataFrame(mission_num, columns=['Mission Number'])
mission_df['Start Time'] = cftime.num2pydate(mission_start_time, units=time_units,
                                             calendar=time_calendar)
mission_df['End Time'] = cftime.num2pydate(mission_end_time, units=time_units,
                                             calendar=time_calendar)
mission_df['Median Time'] = cftime.num2pydate(mission_median_time, units=time_units,
                                             calendar=time_calendar)
mission_df['Number of Waves'] = num_waves_in_mission
mission_df['Offshore Wave Height [m]'] = offshore_wave_height
mission_df['Offshore Mean Period [s]'] = offshore_wave_period
mission_df['Offshore Mean Direction [Degrees From North]'] = offshore_wave_direction

# Sort by most waves in a mission
mission_df.sort_values(by=['Offshore Wave Height [m]'], ascending=False)

,Mission Number,Start Time,End Time,Median Time,Number of Waves,Offshore Wave Height [m],Offshore Mean Period [s],Offshore Mean Direction [Degrees From North]
55,20,2021-10-10 13:07:00,2021-10-10 13:18:14.997300,2021-10-10 13:12:37.498650,2906,2.651632,7.505545,72.586823
48,21,2021-10-10 14:07:00,2021-10-10 14:24:59.995680,2021-10-10 14:15:59.997840,3985,2.545949,7.466452,69.258747
37,22,2021-10-10 15:07:00,2021-10-10 15:15:29.997960,2021-10-10 15:11:14.998980,1775,2.505776,7.528963,69.734439
18,23,2021-10-10 16:11:00,2021-10-10 16:20:59.997600,2021-10-10 16:15:59.998800,1398,2.370009,7.447527,67.902379
63,24,2021-10-11 13:13:00,2021-10-11 13:29:59.995920,2021-10-11 13:21:29.997960,2764,2.162277,7.025174,56.501510
...,...,...,...,...,...,...,...,...
15,3,2021-10-05 17:17:00,2021-10-05 17:36:59.995200,2021-10-05 17:26:59.997600,1153,0.528707,10.033475,84.376313
49,1,2021-10-04 15:00:00,2021-10-04 15:16:59.995920,2021-10-04 15:08:29.997960,1380,0.528284,11.095985,84.508421
64,4,2021-10-05 18:13:00,2021-10-05 18:23:59.997360,2021-10-05 18:18:29.998680,800,0.522822,9.955173,84.789797
57,45,2021-10-18 17:19:00,2021-10-18 17:32:59.996640,2021-10-18 17:25:59.998320,1748,0.515203,5.920682,38.732592


The missions we want to compare will be 18 and 26. They had comparable numbers of waves, similar offshore mean periods and directions but the wave height in mission 26 is 77% larger than mission 18. 